In [1]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs

from django.conf import settings

gemini_api_key = settings.GEMINI_API_KEY

import google.generativeai as genai
genai.configure(api_key=gemini_api_key)


In [40]:
import json
with open('data/002_output_15.json', 'r') as f:
    md15 = json.loads(f.read())
with open('data/002_output_36.json', 'r') as f:
    md41 = json.loads(f.read())

In [74]:
from sapl.materia.models import MateriaLegislativa
from cmj.utils import clean_text
reqs = [34, 36]
materias = MateriaLegislativa.objects.filter(numero__in=reqs, ano=2025, tipo_id=3)

texts = []
for materia in materias:
    doc = pymupdf.open(materia.texto_original.path)
    text = ' '.join([page.get_text() for page in doc])
    text = clean_text(text)
    texts.append(text)


In [75]:
from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
#model = SentenceTransformer('ricardo-filho/bert-base-portuguese-cased-nli-assin-2')

model = SentenceTransformer('sentence-transformers/static-similarity-mrl-multilingual-v1')



#model = SentenceTransformer('neuralmind/bert-large-portuguese-cased')
A_embeddings = model.encode(texts[0])
B_embeddings = model.encode(texts[1])

In [43]:
import numpy as np
result = genai.embed_content(
    model='models/text-embedding-004',
    content=texts,
    task_type='semantic_similarity'
    )
"""A_embeddings = np.array(result['embedding'])

result = genai.embed_content(
    model='models/text-embedding-004',
    content=texts[1],
    task_type='similarity'
    )
B_embeddings = np.array(result['embedding'])"""
A_embeddings = result['embedding'][0]
B_embeddings = result['embedding'][1]

In [76]:
# Define the vectors
A = A_embeddings
B = B_embeddings

# Calculate cosine similarity
cos_similarity = np.dot(A, B) / (np.linalg.norm(A) * np.linalg.norm(B))

# Calculate cosine distance
cos_distance = 1 - cos_similarity

print("Cosine Similarity: {:.2f}".format(cos_similarity))
print("Cosine Distance: {:.2f}".format(cos_distance))

Cosine Similarity: 0.62
Cosine Distance: 0.38


In [77]:
import numpy as np
from scipy.spatial.distance import cosine, euclidean

# Define the vectors
A = A_embeddings
B = B_embeddings

# Calculate cosine distance
cos_distance = cosine(A, B)  # Uses scipy's cosine for cosine distance

print("Cosine Distance: {:.2f}".format(cos_distance))

Cosine Distance: 0.38


In [56]:
print(texts[0])

Requerimento KC-04/2025 À MESA DIRETORA DA CÂMARA MUNICIPAL DE JATAÍ-GO
"Solicita à Secretaria de Obras e de Serviços Urbanos a revitalização da Praça Olho D’água."
A Vereadora Kátia Carvalho, que este subscreve, no uso de suas atribuições legais e regimentais, requer à Presidência desta Casa, após apreciação do plenário, o encaminhamento deste requerimento ao Ilmo. Sr. Cantimiro Melo Martins, Secretário de Obras, e ao Ilmo. Sr. Tiago Alves Rua, Secretário de Serviços Urbanos, solicitando a revitalização da Praça Olho D’água.
JUSTIFICATIVA
Trata-se de demanda recebida em meu gabinete, na qual moradores do município e dos arredores da referida praça se queixam, estando o local em condições de abandono, contendo bancos quebrados, falta de iluminação, parque infantil deteriorado, matagal alto e lixo nos lagos. Também, importante destacar que o lugar tem servido de ponto para uso de entorpecentes, tendo assim, vários usuários semanalmente no local, o que compromete a segurança dos moradore

In [57]:
print(texts[1])

EXCELENTÍSSIMO SENHOR PRESIDENTE DA CÂMARA MUNICIPAL DE JATAÍ, ESTADO DE GOIÁS.
Câmara Municipal de Jataí - GO, 03 de janeiro de 2025. Autor: Vereador Durval Gomes de Oliveira
"Solicita ao Poder Executivo que faça um estudo de viabilidade técnica para realizar a revitalização completa do Parque Ecológico Binômio da Costa Lima (Praça Olho d’água)."
O Parlamentar DURVAL GOMES DE OLIVEIRA, na forma regimental e no uso de suas atribuições legais, requer o encaminhamento de um exemplar deste ao Excelentíssimo Senhor GENEILTON FILHO DE ASSIS, DD. Prefeito Municipal de Jataí e ao Ilustríssimo Senhor CANTIMIRO MELO MARTINS, DD. Secretário de Obras, solicitando ao Poder Executivo que faça um estudo de viabilidade técnica para realizar a revitalização completa do Parque Ecológico Binômio da Costa Lima (Praça Olho d’água).
JUSTIFICATIVA
Ocorre, que o referido parque está completamente abandonado, carecendo de uma revitalização completa por toda sua extensão. As vegetações dentro dos lagos estão a

In [58]:
A

array([ 1.48046874e-02,  5.00745922e-02, -1.14945294e-02,  3.76151018e-02,
        2.94917799e-03,  3.49929705e-02,  3.15210782e-02, -7.82969873e-03,
       -1.69634074e-02, -1.16206203e-02, -1.66373316e-03,  3.53212245e-02,
        4.00100201e-02, -1.12964520e-02, -4.27394398e-02,  8.02392215e-02,
       -6.71935976e-02, -1.96656566e-02, -1.69817661e-03,  4.44402806e-02,
        1.30420998e-02,  2.08235774e-02,  9.35570896e-03, -3.84052657e-02,
        3.41024958e-02,  3.05605438e-02,  1.99299958e-02,  2.82498617e-02,
        2.57530641e-02, -7.31627457e-03,  1.68890562e-02,  1.93675756e-02,
       -3.11751552e-02, -6.54034913e-02,  1.31998481e-02,  3.97277204e-03,
       -3.67867313e-02,  4.59153159e-03,  1.47930225e-02,  3.91750634e-02,
        9.21225548e-03, -1.74524672e-02, -2.94507127e-02, -7.30813434e-03,
       -1.00962203e-02, -2.19934955e-02,  3.25291753e-02, -3.48097016e-03,
       -1.42176645e-02,  2.40785070e-02, -1.54595869e-02, -4.65916730e-02,
        3.53562348e-02,  

In [59]:
B

array([ 1.93122716e-04, -1.82518233e-02, -3.93271856e-02, -1.15938969e-02,
       -1.66277271e-02, -8.24164785e-03,  9.83879319e-04,  1.75859313e-02,
        3.54084335e-02, -1.21890232e-02,  2.09938735e-02,  3.54751199e-02,
        4.36558481e-03, -6.89204782e-03, -2.28934884e-02,  3.86085995e-02,
       -3.74286138e-02,  1.29332487e-02, -8.04015100e-02,  4.44967411e-02,
       -5.08183651e-02,  8.03917646e-03,  3.66058536e-02, -3.38806808e-02,
        5.83434775e-02, -7.18014920e-03,  5.76977292e-03,  5.47018973e-03,
        1.96521413e-02,  3.86791565e-02,  1.94358658e-02,  1.53770503e-02,
        2.11475277e-03, -1.46403182e-02, -4.15778644e-02, -4.11599316e-02,
       -7.50477090e-02,  1.96842644e-02,  6.90467563e-03,  1.86701324e-02,
       -2.52337400e-02, -1.29037974e-02, -4.90945242e-02, -4.24623042e-02,
       -5.33350650e-03, -5.00404462e-02, -1.68250483e-02,  3.67825106e-02,
       -8.60154554e-02,  1.06413616e-02,  4.99596074e-02, -2.88789347e-02,
        3.51296142e-02,  